In [77]:
import ee
import numpy as np
import pandas as pd

from earthshot import mon_stats
from earthshot import water_viz as vis
from earthshot import normalize as norm
from earthshot import water_common as common
from statistics import mean

import geemap.eefolium as geemap
import folium
from folium import plugins

import matplotlib.pyplot as plt
%matplotlib inline

In [78]:
ee.Initialize()

In [84]:
MAR_projects = ee.FeatureCollection('users/amgadellaboudy/Global_MAR_Inventory')

Vegetation_index = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY")

MAR_1982 = MAR_projects.filter('year_opera > 1980').filter('year_opera < 2020') #filter MAR projects that were built between 1982 and 2019

In [85]:
NDVI_years = ee.ImageCollection('users/amgadellaboudy/EVI_year')
NDVI_2020 = (Vegetation_index.filter(ee.Filter.date('2020-01-01', '2020-12-01'))
            .select('volumetric_soil_water_layer_1').reduce(ee.Reducer.mean()))

for i in range(1981,2020):
    NDVI_image = (Vegetation_index.filter(ee.Filter.date(('{}-01-01'.format(str(i))), ('{}-12-01'.format(str(i)))))
    .select('volumetric_soil_water_layer_1').reduce(ee.Reducer.mean())) #Take vegetation index for given year, average values into one image
    NDVI_diff = NDVI_2020.subtract(NDVI_image).set({'Year': i}) #Store NDVI difference into image, set year as property
    NDVI_years = NDVI_years.merge(NDVI_diff) #Collect into image collection

    
    

In [86]:
#MAR_2001 = MAR_2000.filter(ee.Filter.eq('year_opera', 2001))  #Initialize MAR Feature Collection for year 2001
NDVI_1982 = NDVI_years.filter(ee.Filter.eq('Year', 1981)).select('volumetric_soil_water_layer_1_mean').first() #Take image for NDVI diff for 1982
NDVI_1982_feat = NDVI_1982.sampleRegions(collection = MAR_1982, properties = ['year_opera'], scale = 6000, geometries = True) #Initiate image collection for EVI diff at MAR locations
 
#for i in range(2002,2020):
  #  MAR_year = MAR_2000.filter(ee.Filter.eq('year_opera', i))
  #  EVI_diff = EVI_year.filter(ee.Filter.eq('Year', i)).select('EVI_mean').reduce(ee.Reducer.first())
   # EVI_feat = EVI_diff.sampleRegions(collection = MAR_year, properties = ['year_opera'], scale = 250, geometries = True)
   # EVI_2001_feat.merge(EVI_feat)

In [87]:
print(NDVI_1982_feat.filter('volumetric_soil_water_layer_1_mean > 0').size().getInfo())

226


In [88]:
print(NDVI_1982_feat.size().getInfo())

478


In [43]:
the_map = geemap.Map()

vis_params = {
    'min': -1000, 'max': 1000, 'dimensions': 512,
    'palette': ['Red', 'Yellow', 'Blue']}

the_map.addLayer(NDVI_1982, vis_params, 'EVI_diff')
the_map.addLayer(MAR_1982.draw(color= '006600', pointRadius = 5, strokeWidth= 5), {}, name = 'MAR locations')

vis.folium_display(the_map)

In [108]:
xx = ee.batch.Export.image.toAsset(
        EVI_img,
        description = 'EVI_export',
        region = bbox_global,
        assetId = 'users/amgadellaboudy/EVI_img',
        scale = 250,
        maxPixels = 100000000000).start()